In [130]:
from Bio import AlignIO, SeqIO, Seq, pairwise2
from Bio.pairwise2 import format_alignment
import pandas as pd

In [131]:
min_length_aligned_sequence = 30 #Minimal lenght of sequence which could be an exon
extreme_homology = 0.97 #percentage of homology of sequence, treshold #I assume two faulty aligned nucleotides per 100 (98%) and one more nt because sometimes latest nt can move from end of one sequence to beginning next sequence
middle_homology = 0.9 #nothing at the moment

In [132]:
path11 = './alingment_gen_transkrypt//EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05.aln'
path12 = './alingment_gen_transkrypt//EG_OG0030308_t2_g2_EG_PAM1_GP153_OV05.aln'
path13 = './alingment_gen_transkrypt//EG_OG0030309_t3_g3_EG_PAM1_GP153_OV05.aln'
path21 = './alingment_gen_transkrypt//EH_OG0030307_t1_g1_EH_PAM1_GP153_OV05.aln'
path22 = './alingment_gen_transkrypt//EH_OG0030308_t2_g2_EH_PAM1_GP153_OV05.aln'
path23 = './alingment_gen_transkrypt//EH_OG0030309_t3_g3_EH_PAM1_GP153_OV05.aln'
path31 = './alingment_gen_transkrypt//EL_OG0030307_t1_g1_EL_PAM1_GP153_OV05.aln'
path32 = './alingment_gen_transkrypt//EL_OG0030308_t2_g2_EL_PAM1_GP153_OV05.aln'
path33 = './alingment_gen_transkrypt//EL_OG0030309_t3_g3_EL_PAM1_GP153_OV05.aln'

In [154]:
def cutting_scrap_right(plik, acceptable_gap_length):
    
    gaps_signs = "-" * acceptable_gap_length #maximum length of gaps one by one in sequence
    
    alignment = AlignIO.read(plik, "clustal") #loading data witch "clustal" extension 
    
        
        
    while True:
        first_nucleotides_pair = alignment[:, 0]#first nucleotides in both strands as variable
        if "-" in first_nucleotides_pair: #deleting mismatches at the start #do tej pory wszystko git. program odnajduje znak "-" w pierwszej. parze
            alignment = alignment[:, 1:]
            #print(f"alignment po obrobce:\n{alignment}") #chekpoint
            continue
    
            
        
        #changing extension
        seq1 = alignment[0].seq #from whole variable choose only .seq #upper strand
        seq2 = alignment[1].seq #lower strand

        
        #Counting distance:
        first_gaps_position_seq1 = seq1.find(gaps_signs) #getting index of first gap. There is also included gap in genomic sequence - made it intentionally
        first_gaps_position_seq2 = seq2.find(gaps_signs)  #this code prints alignment since 1st to certain number (without gaps at the end!)
        #print(f"first gap position in first sequence:{first_gaps_position_seq1}, in 2nd sequence: {first_gaps_position_seq2}") #checkpoint
        
        if first_gaps_position_seq2 < 0:
            distance_between_first_nt_and_gap = first_gaps_position_seq1
        else:
            distance_between_first_nt_and_gap = min(first_gaps_position_seq1, first_gaps_position_seq2)

        
        local_alignment = pairwise2.align.localxx(seq1[:distance_between_first_nt_and_gap], seq2[:distance_between_first_nt_and_gap], \
                                                    one_alignment_only = True)#score_only = True print only score
        #I choose function parameters like "xx" what means no gap penalty while opening gaps or longering them and no penalties for mismatch. Just pure score of alignment to count homology
        #print("local_alignment:", local_alignment) #checkpoint
       
        try: #obsluga wyjatkow
            local_homology_percentage = (local_alignment[0].score / distance_between_first_nt_and_gap) # that sign "[]" becouse local_alignment's type is list
        except IndexError:
            #print("index error", len(seq1))
            if len(seq1) < 100:
                print(f"\n \n {alignment[0].id} sequence is too short or sequence has not exons. \n")
                break
            continue
        
        
        if local_homology_percentage < extreme_homology:
            #print(f"\n Too low score of homology. \n alignment: {alignment[:, :50]} \n score: {local_alignment[0].score}, distance: {distance_between_first_nt_and_gap}, percent: {round(local_homology_percentage, 2)*100}%, \n {local_alignment}")
            alignment = alignment[:, distance_between_first_nt_and_gap:] 
            
        else:
            if distance_between_first_nt_and_gap < min_length_aligned_sequence:
                #print(f"\n alignment is too short. \n distance between sequences {distance_between_first_nt_and_gap}, \n alignment: {alignment}, \n score: {local_alignment[0].score}, distance: {distance_between_first_nt_and_gap}, procentowo: {round(local_homology_percentage, 2)*100}%, \n {local_alignment}")
                alignment = alignment[:, distance_between_first_nt_and_gap:]
            else:
                #print(f"Sequence {alignment[0].id} cut properly from LEFT side")
                #print(f"""\n \n \n victory lewo! \n distance between sequences: {distance_between_first_nt_and_gap},
                #alignment: {alignment[:, :50]}, 
                #score: {local_alignment[0].score}, dystans: {distance_between_first_nt_and_gap}, 
                #percent: {round(local_homology_percentage, 4)*100}%, \n \n local alignment to confirm: \n{local_alignment}""")
                break
        

    
######################### TERAZ OD PRAWEJ DO LEWEJ #######################  

    
    while True:
        last_nucleotides_pair = alignment[:, -1]#last nucleotides in both strands as variable
        if "-" in last_nucleotides_pair: #deleting mismatches at the end
            alignment = alignment[:, :-1]
            #print(f"alignment check:\n{alignment[:, -50:-1]}") checkpoint
            continue
                
            
#tu jest problem - funkcja usuwa o jedną pare nukleotydow za duzo, mimo ze nie są ona gap "-". - zachodzi jedna dodatkowa iteracja Na razie pomijam
        #jak nic nie podziała to trzeba bedzie zmierzyc odleglosc od ostatniego nukleotydu do pierwszego (szerokosc gapow) i usunac to w petli for
        
        #changing extension
        seq1 = alignment[0].seq #from whole file choose only .seq #upper strand
        seq2 = alignment[1].seq #lower strand
        
       
        last_gaps_position_seq1 = seq1.rfind(gaps_signs)  #Counting distance: #getting index of first gap. There is also included gap in genomic sequence - made it intentionally
        last_gaps_position_seq1 = last_gaps_position_seq1 + acceptable_gap_length
        
        last_gaps_position_seq2 = seq2.rfind(gaps_signs)  #this code prints alignment since 1st to certain number (without gaps at the end!)
        last_gaps_position_seq2 = last_gaps_position_seq2 + acceptable_gap_length #now we have to add length of gap to index, because python starts counting from 0 nt, and we just change direction of counting.
        #print("seq1:", last_gaps_position_seq1, "seq2:", last_gaps_position_seq2) checkpoint
        
        
        if last_gaps_position_seq2 < 0: #it means, if there is no gaps in second sequention, gap position is gap position in seq1. Additional, rfind() function return "-1" if it won't find any gaps in seq
            distance_between_first_nt_and_gap = (len(seq1) - last_gaps_position_seq1)
            distance_between_first_nt_and_gap = distance_between_first_nt_and_gap - 1 #correction to counting from 0th index
            #print("pierwszy", distance_between_first_nt_and_gap)
        else:
            distance_between_first_nt_and_gap = (len(seq1) - max(last_gaps_position_seq1, last_gaps_position_seq2))#tutaj napisalem taki kod, ktory wybiera najmniejsza odleglosc miedzy pierwszym nukleotydem a najblizsza podwojna przerwa. Ale to moze uwzglednic rowniez przerwe w sekwencji genowej - tak zostawiam
            #print("drugi", distance_between_first_nt_and_gap)
            
            
            local_alignment = pairwise2.align.localxx(seq1[-distance_between_first_nt_and_gap:], seq2[-distance_between_first_nt_and_gap:], \
                                                    one_alignment_only = True) #score_only = True print only score #I choose function parameters like "xx" what means no gap penalty while opening gaps or longering them and no penalties for mismatch. Just pure score of alignment
        #print("local_alignment:", local_alignment) #checkpoint
        #mam wrazenie ze python nie chce mi lapac ostatniego nukleotydu w komendzie: [liczba:] - zrobil alignment dla 180nt gdzie dlugosc gapu to 180

        

        try: #obsluga wyjatkow
            local_homology_percentage = (local_alignment[0].score / distance_between_first_nt_and_gap) #that sign "[]" becouse local_alignment's type is list
        except IndexError:
            if len(seq1) < 100:
                print(f"\n \n {alignment[0].id} sequence is too short or sequence has not exons. \n")
                break
            continue
            #niestety ten warunek jakoś nie działa na sekwencje 32

        
        if local_homology_percentage < extreme_homology:
            #print(f"\n Alignment ma zbyt niska homologie. \n alignment: {alignment[:, -50:]} \n score: {local_alignment[0].score}, \dystans: {distance_between_first_nt_and_gap}, procentowo: {round(local_homology_percentage, 2)*100}%, \n {local_alignment}")
            alignment = alignment[:, :-distance_between_first_nt_and_gap] 
            
        else:
            if distance_between_first_nt_and_gap < min_length_aligned_sequence:
                #print(f"\n alignment jest za krotki. \n dystans miedzy sekwencjami {distance_between_first_nt_and_gap}, \n alignment: {alignment}, \n score: {local_alignment[0].score}, dystans: {distance_between_first_nt_and_gap}, procentowo: {round(local_homology_percentage, 2)*100}%, \n {local_alignment}")
                alignment = alignment[:, :-distance_between_first_nt_and_gap]
            else:
                #print(f"Sequence: {alignment[0].id} cut properly from RIGHT side")
                #print(f"""\n \n \n victory prawo! \n distance between sequences {distance_between_first_nt_and_gap},
                #alignment: {alignment[:, -50:]}, 
                #score: {local_alignment[0].score}, distance: {distance_between_first_nt_and_gap}, 
                #percent: {round(local_homology_percentage, 4)*100}%, \n \n local alignment to confirm: \n{local_alignment}""")
                break

    index = 0
    id = []
    is_it_intron = []
    class_of_exon = []
    exon_or_intron_number = []
    percent_of_homology_for_exons = []
    length = []
    first_10_nt = []
    last_10_nt = []
    sequence = []
    path = []
    
    while True: #chyba mimo wszystko zostajemy przy while true bo trudno jest mi wyciagnac liczbe plikow ktore beda lacznie wprowadzane. Tak jest chyba łatwiej
        #problem taki wynikajacy ze specyfikacji programu jest taki, że outputowa tabelka bedzie tylko dla jednego organizmu. Potem mozna je jakos polaczyc w calosc
        try: #obsluga wyjatkow
            local_homology_percentage = (local_alignment[0].score / distance_between_first_nt_and_gap) #that sign "[]" becouse local_alignment's type is list
        except IndexError:
            if len(seq1) < 100:
                print(f"\n \n {alignment[0].id} sequence is too short or sequence has not exons. \n")
                break
            continue
        #chwila chwila, czy w tym momencie jest nam wogole potrzebna ta petla while true? bo ona tylko opetla jedna petle ktora sama sie wygasza po spelnieniu warunków
        nucleotides = ["a", "t", "g", "c", "A", "T", "G", "C"]

        #plan jest taki: napierdalamy z karabinkow ile wzlezie
        #plan jest taki: szukamy 1nt eksonu, zrobimy warunek ze jesli wynikiem alignment_of_exon bedzie NIC (tak jak tearz) to puszczamy alignment na intronie dopisujac tylko index i sekwencje druga
        #EKSON INTRON EXON INTRON EXON INTRON
        #dla pierwszego eksonu robimy alignment i go zapisujemy do df
        #ucinamy ten ekosn tak jak wczesniej smieci (wyzej)
        #robimy alignment dla intronu i go ucinamy
        #mozemy to zapetlic az do momentu gdy dlugosc seq1 bedzie = 0

        #ewentualnie nie trzeba wcale tego przycinać i eliminowac bo mozna przesuwac obszar wyszukiwania przez funckje find() podajac jej miejsce startu i stopu. Ale w sumie to na jedno wychodzi. Zalezy co jest szybsze dla komputera
        #string.find(value, start, end) #ale musielibysmy i tak tworzyc dwie odnogi zeby wklepac jej raz wzor szukania "--"        a drugim razem ktorys z nukleotydow
        

        if len(seq1) > 1:
            #print("Uruchomienie petli")
            #sequence1 = alignment[0].seq #musialem dodac te dwie linijki bo ucinanie tego intronu nie wychodzilo poza ta petle. Nie uzylem wczesniej napisanej funkcji zeby uniknac bledu jakiegos
            #sequence2 = alignment[1].seq
            if seq1[0] != gaps_signs: #mining data from exons   
                #print("uruchomienie analizy eksonow")
                index += 1
                last_nt_exon_index = seq1.find(gaps_signs) #zatrzymuje sie gdy znajdzie gapa - dwa gapy
                
                if last_nt_exon_index == -1: #it means if it is last exon
                    last_nt_exon_index = len(seq1)
                    
                
                alignment_of_exon = pairwise2.align.localxx(seq1[:last_nt_exon_index], seq2[:last_nt_exon_index], one_alignment_only = True) #zdaje sobie sprawe ze score = 94 gdy dlugosc = 96, czyli homologia < 97%. Ale to dlatego ze funkcja wprowadza swoje wlasne gapy, ktore i tak nie maja znaczenia bo liczy sie tylko dopasowanie czyli poziom identycznych nukleotydow
                #alignment = alignment[:, last_nt_exon_index:] #nie zadziala bo tutaj elementem przycinanym powinna być seq1

                #counting percent of homology
                percent_of_homology = alignment_of_exon[0].score / last_nt_exon_index
                #print(percent_of_homology)
                
                #print("zakonczenie analizy eksonow")
                #print("score alignmentu = ", alignment_of_exon[0].score)
                #print("seq1 = ", seq1)
                #print("alignment = ", alignment_of_exon[0])
                

                #TWORZENIE TABELKI
                id.append(alignment[0].id)
                exon_or_intron_number.append(index)
                is_it_intron.append(False)
                percent_of_homology_for_exons.append(round(percent_of_homology * 100, 2))#tylko dla exonow
                length.append(last_nt_exon_index)
                first_10_nt.append(str(seq1[:10]))
                last_10_nt.append(str(seq1[last_nt_exon_index-10 : last_nt_exon_index])) #to jest 10 ostatnich nukleotydow sekwencji calej, nie alignmentu
                sequence.append(str(seq1[:last_nt_exon_index])) #odblokuj ta opcje w przyszlosci! #tylko dla exonow!
                path.append(str(plik))
                
                if percent_of_homology >= 0.9:
                    class_of_exon.append("1")
                elif percent_of_homology < 0.9 and percent_of_homology > 0:
                    class_of_exon.append("2")
                else:
                    class_of_exon.append("3")
                
                seq1 = seq1[last_nt_exon_index:]
                seq2 = seq2[last_nt_exon_index:] #BINGO tym skracamy seq1
                #print("zakonczenie analizy eksonow")

                

            if len(seq1) > 1 and seq1[0] == "-":
                index += 1
                #print("uruchomienie analizy intronow")
                last_nt_intron_index = None #searching for first nt in sequence (a, t, g, or c)
                for nt in nucleotides:
                    nt_index = seq1.find(nt)
                    if nt_index != -1:
                        if last_nt_intron_index is None or last_nt_intron_index > nt_index:
                            last_nt_intron_index = nt_index -1
                            break #bede musial to skontrolowac, cyz przypadkiem ten break nie hamuje calej funkcji przed znalezieniem najmniejszego indeksu
                    else:
                        break
                if last_nt_intron_index == seq2[-1]:
                    print("Cutting alignment from the end was faulty!")
                
                
                id.append(alignment[0].id)
                exon_or_intron_number.append(index)
                is_it_intron.append(True) #czy te cudzyslowy sa potrzebne? czy to bedzie traktowane jako string czy jako true or false?
                percent_of_homology_for_exons.append(None)#tylko dla exonow
                length.append(last_nt_intron_index)
                first_10_nt.append(str(seq2[:10]))
                last_10_nt.append(str(seq2[last_nt_intron_index-10 : last_nt_intron_index]))
                sequence.append(str(seq2[:last_nt_intron_index])) #odblokuj ta opcje w przyszlosci! #tylko dla exonow!
                path.append(str(plik))
                class_of_exon.append(None)

                seq1 = seq1[last_nt_intron_index:]
                seq2 = seq2[last_nt_intron_index:]
                #print("zakonczenie analizy intronow")
            else:
                break
                

                
    dictionary = {"ID":id, "exon_or_intron_number":exon_or_intron_number, "is_it_intron":is_it_intron, "class_of_exon":class_of_exon, "percent_of_homology_for_exons":percent_of_homology_for_exons, "length":length, "first_10_nt" : first_10_nt, "last_10_nt" : last_10_nt, "path":path, "sequence" : sequence}
    #print(dictionary)        
    data_frame = pd.DataFrame(dictionary) #ramke mamy dobrze zrobiona, niestety ona na razie pobiera dane z tylko jednego pliku wejsciowego i z tylko pierwszego eksonu
    print()
    print()
    print("\n\n\n\n\n", data_frame, "\n\n\n\n\n")

        
        
        #fragment zapisujacy wartosci do ramki
#uładnij kod, usun zbedne komentarze, dodaj te niezbedne po angielsku 

#na razie ta funkcja dziala tak: jedna sciezka, jedna ramka. Musze zmienic to tak by wiele sciezek = jedna ramka






   


In [155]:
#cutting_scrap_right(path11, 2)
#cutting_scrap_right(path12, 2)
#cutting_scrap_right(path13, 2)
#cutting_scrap_right(path21, 2)
#cutting_scrap_right(path22, 2)
#cutting_scrap_right(path23, 2)
#cutting_scrap_right(path31, 2)
#cutting_scrap_right(path32, 2)
#cutting_scrap_right(path33, 2)

In [156]:
import os
directory = './alingment_gen_transkrypt'

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if not os.path.isfile(f):
        continue
    if f.endswith(".aln"):
        #print("sciezka: \t", f)
        cutting_scrap_right(f, 2)
        print("\n\n\n\n")
    
    








                                       ID  exon_or_intron_number  is_it_intron  \
0  EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                      1         False   
1  EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                      2          True   
2  EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                      3         False   
3  EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                      4          True   
4  EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                      5         False   
5  EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                      6          True   
6  EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                      7         False   

  class_of_exon  percent_of_homology_for_exons  length first_10_nt  \
0             1                          97.92      96  ctacacgtac   
1          None                            NaN     102  gcccgccgcc   
2             1                         100.00     145  gatcgagctg   
3          None                            NaN     237  gaccggg

Pierwsza decyzja w przypadku poprawiania kodu na jego prawy odpowiednik (reversed):
- nazwy zmiennych które jawnie odnoszą się do pozycji, np. lewe coś tam, pierwszy nukleotyd zostały zmienione na ich właściwe odpowiednki
- Pozostałe elementy zostaja spojne - alignment to wciaz zmienna alignment pojawiajaca sie w obu fragmentach kodu.

Druga decyzja:
- kod bedzie mial strukture:
      - przycinanie lewej strony
      - przycinanie prawej strony
      - wysylka do excela
  - i żaden z tych fragmentow nie 